In [15]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
import time 
import numpy as np
import pandas as pd
items=[]
index=0
result = np.array([])
i = 0
def getTable(id):
    global result, i
    setSelect('ctl00_ContentPlaceHolder1_ddlInstitutes',f"{id}")
    setSelect('ctl00_ContentPlaceHolder1_ddlDepartment','ALL')
    setSelect('ctl00_ContentPlaceHolder1_ddlMEProgram','ALL')
    setSelect('ctl00_ContentPlaceHolder1_ddlGroupList','ALL')

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_gvMapping'})
            
    if table:
        if i == 0:
            result = np.array([th.text.strip() for th in table.find_all('th')])
        rows = []
        for tr in table.find_all('tr')[1:]:
            cells = [td.text.strip() for td in tr.find_all('td')]
            rows.append(cells)
        
        result = np.vstack([result, rows]) if result.size else np.array(rows)
        i += 1
        # print(i) progess tracker

def setSelect(selectId,optionValue):
    WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, selectId))
        )
    driver.execute_script(f"document.getElementById('{selectId}').style.display='block';")

    dropdown = Select(driver.find_element(By.ID, selectId))


    # print(optionValue)
    dropdown.select_by_value(optionValue)

def scrap(url,driver):
    instituteIds=[]
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('select', {'id': 'ctl00_ContentPlaceHolder1_ddlInstitutes'})
    for x in table.find_all()[1:]:
        instituteIds.append(x.attrs['value'])
    driver.get(url)
    result = np.array([])
    for id in instituteIds:
        getTable(id)



service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

urls=["https://admissions.nic.in/CCMT/Applicant/Report/seatmapping.aspx?enc=Nm7QwHILXclJQSv2YVS+7jwDCTS6FhNmjKGAxsSogZ/WSC5YMGOlm6wjUQsaRQ5B"]

for url in urls:
    scrap(url,driver)
driver.quit()
df = pd.DataFrame(result)
df.to_csv('CCMT_data.csv', index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
